In [ ]:
##jsonl - csv

In [ ]:
!pip install pymysql sqlalchemy

In [ ]:
import pandas as pd
import json
import os
import glob 


input_folder_path = r'C:\Users\Rohan\Downloads\fermbile\Datasets-2025-08-08'

output_csv_file = 'full_news_events_data_combined.csv'

all_flattened_data = []


def safe_extract(record, keys):
    current = record
    for key in keys:
        if isinstance(current, dict) and key in current:
            current = current[key]
        else:
            return None
   
    if isinstance(current, list):
        if all(isinstance(item, (str, int, float)) for item in current):
            return " | ".join(map(str, current))
        
        elif keys[-1] == 'location_data':
            locations = []
            for loc in current:
                parts = [loc.get('city'), loc.get('country')]
                locations.append(", ".join(filter(None, parts)))
            return " | ".join(locations)
       
        else:
            try:
               
                return json.dumps(current, ensure_ascii=False) 
            except:
                return str(current)
    return current


jsonl_files = glob.glob(os.path.join(input_folder_path, '*.jsonl'))

if not jsonl_files:
    print(f" No .jsonl files found in the directory: {input_folder_path}")
else:
    print(f"Processing {len(jsonl_files)} .jsonl files...")

    
    for file_name in jsonl_files:
        print(f"   Reading {file_name}...")
        data = []
        
        
        try:
            
            with open(file_name, 'r', encoding='utf-8') as f:
                for line in f:
                    
                    if line.strip():
                        data.append(json.loads(line))
        except Exception as e:
            print(f"    Error reading JSONL file {file_name}: {e}. Skipping file.")
            continue

        #  Extract and flatten the data from the current file
        for record in data:
            try:
                events = record.get('data', [])
                
                for event in events:
                    if event.get('type') == 'news_event':
                        attributes = event.get('attributes', {})
                        relationships = event.get('relationships', {})
                        
                        # Extract event attributes
                        row = {
                            'event_id': event.get('id'),
                            'event_category': attributes.get('category'),
                            'event_summary': attributes.get('summary'),
                            'effective_date': attributes.get('effective_date'),
                            'found_at': attributes.get('found_at'),
                            'location': attributes.get('location'),
                            'award': attributes.get('award'),
                            'recognition': attributes.get('recognition'),
                            'product': attributes.get('product'),
                            'article_sentence': attributes.get('article_sentence'),
                            'source_file': os.path.basename(file_name) # Add source file for traceability
                        }

                       
                        company_id = safe_extract(relationships, ['company1', 'data', 'id'])
                        if company_id:
                            
                            company_info = next((item['attributes'] for item in record.get('included', []) 
                                                 if item.get('id') == company_id and item.get('type') == 'company'), {})
                            row['company_name'] = company_info.get('company_name')
                            row['company_domain'] = company_info.get('domain')
                            row['company_ticker'] = company_info.get('ticker')
                        else:
                            row['company_name'] = None
                            row['company_domain'] = None
                            row['company_ticker'] = None

                       
                        article_id = safe_extract(relationships, ['most_relevant_source', 'data', 'id'])
                        if article_id:
                           
                            article_info = next((item['attributes'] for item in record.get('included', []) 
                                                 if item.get('id') == article_id and item.get('type') == 'news_article'), {})
                            row['article_title'] = article_info.get('title')
                            row['article_url'] = article_info.get('url')
                        else:
                            row['article_title'] = None
                            row['article_url'] = None
                        
                        all_flattened_data.append(row)

            except Exception as e:
                
                continue

   
    if all_flattened_data:
        df = pd.DataFrame(all_flattened_data)
        
        
        final_columns = [
            'event_id', 'source_file', 'company_name', 'company_domain', 'company_ticker',
            'event_category', 'event_summary', 'effective_date', 'found_at',
            'location', 'award', 'recognition', 'product', 'article_title', 'article_url',
            'article_sentence'
        ]
        
       
        df = df.reindex(columns=final_columns)

        
        df.to_csv(output_csv_file, index=False, encoding='utf-8')
        print(f"\n Successfully processed {len(jsonl_files)} files and saved {len(df)} events to **{output_csv_file}**.")
    else:
        print("\n No news events were successfully extracted from any of the files.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.parse import urlparse
from sqlalchemy import create_engine


pd.set_option('display.max_columns', None)

In [ ]:
##loading data

df = pd.read_csv(r"C:\Users\Rohan\Downloads\fermbile\full_news_events_data_combined.csv")
print("\n Data Loaded Successfully!")
print(f"Total Rows: {df.shape[0]}, Columns: {df.shape[1]}")
df.head()

In [ ]:
##summary

def profile_dataframe(df):
    summary = pd.DataFrame({
        'dtype': df.dtypes,
        'missing_count': df.isna().sum(),
        'missing_%': (df.isna().mean() * 100).round(2),
        'unique_count': df.nunique()
    })
    return summary

profile_summary = profile_dataframe(df)
print("\n Data Profiling Summary:")
profile_summary

In [ ]:
##Data quality check

def is_valid_url(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except:
        return False

def check_data_quality(df):
    dq_report = {}

    dq_report['missing_values_%'] = df.isna().mean().mean() * 100
    dq_report['duplicate_rows'] = df.duplicated().sum()

    if 'article_url' in df.columns:
        invalid_urls = df['article_url'].apply(lambda x: not is_valid_url(str(x)) if pd.notnull(x) else False).sum()
        dq_report['invalid_urls'] = invalid_urls

    return dq_report

dq_before = check_data_quality(df)
print("\n Data Quality Issues (Before Cleaning):")
print(dq_before)

In [ ]:
##Data cleaning

clean_df = df.copy()


clean_df.drop_duplicates(inplace=True)


for col in clean_df.select_dtypes(include='object').columns:
    clean_df[col] = clean_df[col].astype(str).str.strip()


essential_cols = [col for col in ['title', 'article_url'] if col in clean_df.columns]
clean_df.dropna(subset=essential_cols, inplace=True)


if 'article_url' in clean_df.columns:
    clean_df['article_url'] = clean_df['article_url'].apply(lambda x: str(x).strip().lower().rstrip('/'))


In [ ]:
##Data quality summary

dq_after = check_data_quality(clean_df)

print("\n Data Quality Comparison:")
comparison = pd.DataFrame([dq_before, dq_after], index=['Before', 'After']).T
comparison


In [ ]:
##Visualization of Data Quality

missing = df.isna().mean() * 100
plt.figure(figsize=(10,4))
missing.sort_values(ascending=False).plot(kind='bar', color='tomato')
plt.title('Percentage of Missing Values by Column')
plt.ylabel('% Missing')
plt.show()

plt.figure(figsize=(6,4))
sns.barplot(x=comparison.columns, y=comparison.loc['missing_values_%'])
plt.title('Missing Data Before vs After Cleaning')
plt.show()


In [ ]:
clean_df.to_csv('Cleaned_data.csv', index=False)
print("\n Cleaned data saved as 'Cleaned_data.csv'")

In [ ]:


from sqlalchemy import create_engine


mysql_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '########',
    'database': 'news_events_db'
}


conn_str = (
    f"mysql+pymysql://{mysql_config['user']}:{mysql_config['password']}@"
    f"{mysql_config['host']}/{mysql_config['database']}"
)


engine = create_engine(conn_str)
print(" MySQL connection engine created successfully!")


In [ ]:
# Upload cleaned data to MySQL
try:
    clean_df.to_sql(
        name='news_events_cleaned',   # table name
        con=engine,                   # connection engine created earlier
        if_exists='replace',          # overwrite existing table (use 'append' if you want to add)
        index=False                   # don't write pandas index
    )
    print(" Cleaned data uploaded to MySQL successfully!")

except Exception as e:
    print(" Upload failed:", e)


In [ ]:
dq_summary = {
    'run_id': '001',
    'rows_before': len(df),
    'rows_after': len(clean_df),
    'duplicates_removed': len(df) - len(clean_df),
}

dq_df = pd.DataFrame([dq_summary])
dq_df.to_sql('dq_metrics_run', con=engine, if_exists='append', index=False)
print(" Metrics inserted into dq_metrics_run!")
